In [212]:
#Cleaning to-dos:
#clean up mocktest()
#write docstrings
#remove confusing comments


from astropy.coordinates import ICRS
from astropy import units as u
from astropy.coordinates.angles import Angle
from astropy.coordinates import match_coordinates_sky
import numpy as np
import os
import sys
import pandas as pd

class OpenCluster():

	'''
	Open cluster class.
	'''

	def __init__(self, name, sampedro_name, radius=0., age=0., distance=100.):
		
		#prompt and file specific cluster name
		self.name=name
		self.sampedro_name=sampedro_name
		#radius in acrmin
		self.radius=radius
		#age in Gyr
		self.age=age
		#distance in pc
		self.distance=distance
		#subcats with different numbers of membership allocations
		self.sampedro=[]
		self.sampedro_n1=[]
		self.sampedro_n2=[]
		self.sampedro_n3=[]
		#Pan-STARRS catalog
		self.PS=[]
		#K2 EVEREST search for the cluster 
		self.K2=[]
		#subcat of self.K2 which has 2MASS IDs
		self.K2MASS=[]
  
	#----------------------------------------------------------------------------------------------

	def loadcatalogs(self,debug=False):
        
		path="/work1/eilin/data/CLUSTERS_01/k2-panstarrs-sampedro"
		os.chdir(path)
#SAMPEDRO
		self.sampedro = pd.read_csv(path+'/cats/'+self.sampedro_name+'_Sampedro_cluster_members_query.csv',
                                       usecols=[2,3,5,57,58,59],
                                       delimiter='\t',
                                      )

		self.sampedro.RAJ2000 = to_xms(self.sampedro.RAJ2000, 'udeg')
		self.sampedro.DEJ2000 = to_xms(self.sampedro.DEJ2000, 'udeg')   
#PanSTARRS:
		self.PS = pd.read_csv(path+'/cats/'+self.sampedro_name+'_panstarrs_search.txt',
                                       usecols=['objID','raMean','decMean','gMeanPSFMag','rMeanPSFMag','iMeanPSFMag','zMeanPSFMag','yMeanPSFMag'],
                                       skiprows=[1],
                                       delimiter='\t',
                                      )
		self.PS.raMean = to_xms(self.PS.raMean, 'h')
		self.PS.decMean = to_xms(self.PS.decMean, 'd')          
#K2
		self.K2 = pd.read_csv(path+'/cats/'+self.sampedro_name+'_k2_search.txt',
                                       usecols=['K2 ID','RA (J2000)','Dec (J2000)','2MASS ID'],
                                       skiprows=[1],
                                       delimiter='\t',
                                      )
		self.K2.rename(index=int,
                       columns={'K2 ID':'EPIC','RA (J2000)':'RAJ2000','Dec (J2000)':'DEJ2000','2MASS ID':'2MASS'},
                       inplace=True)
		self.K2.RAJ2000 = to_xms(self.K2.RAJ2000, 'h')
		self.K2.DEJ2000 = to_xms(self.K2.DEJ2000, 'd') 
#K2MASS
		self.K2MASS = self.K2[self.K2['2MASS'].notnull()]
        
		if debug == True:
			print('Currently working in \"' + os.getcwd()+ '\"\n')
			print('These are our K2 data:\n\n{}\n'.format(self.K2.head()))
			print('These are our Sampedro data:\n\n{}\n'.format(self.sampedro.head()))
			print('These are our Pan-STARRS data:\n\n{}\n'.format(self.PS.head()))
			print('\nNumber of Everest LCs: \n{}\n'.format(len(self.K2)))
			print('\nNumber of Everest LCs with 2MASS ID: \n{}\n'.format(len(self.K2MASS)))		
            
		return
    
	def refinesampedro(self,debug=False):
        
		self.sampedro_n1=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==1]
		self.sampedro_n2=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==2]
		self.sampedro_n3=self.sampedro[(self.sampedro.ClassM1+self.sampedro.ClassM2+self.sampedro.ClassM3)==3]

		if debug==True:
			print('These are single validation members:\n\n{}\n'.format(self.sampedro_n1.head()))
			print('These are double validation members:\n\n{}\n'.format(self.sampedro_n2.head()))
			print('These are triple validation members:\n\n{}\n'.format(self.sampedro_n3.head()))
		
		return
    
def to_xms(series, x):
    
    list_ = series.tolist()
    if x == 'h':
        series = [item[:2]+'h'+item[3:5]+'m'+item[6:]+'s' for item in list_]
    elif x == 'd':
        series = [item[:3]+'d'+item[4:6]+'m'+item[7:]+'s' for item in list_]
    elif x == 'udeg':
        list_=Angle(list_,unit=u.deg)
        series = Angle.to_string(list_,sep='dms',pad=True)
    return series





In [213]:

x=OpenCluster('M44','M44', radius=47, age=0.73, distance=181.5)
x.loadcatalogs(debug=False)
x.refinesampedro(debug=True)       

These are single validation members:

        UCAC4       RAJ2000       DEJ2000  ClassM1  ClassM2  ClassM3
0  546-046614  129d43m44.4s  19d11m53.88s        0        1        0
1  546-046624  129d47m09.6s   19d11m29.4s        0        1        0
4  546-046634  129d48m57.6s  19d11m20.76s        0        1        0
5  546-046639  129d50m56.4s  19d09m41.04s        0        1        0
6  546-046640    129d51m00s  19d07m40.44s        0        1        0

These are double validation members:

          UCAC4       RAJ2000       DEJ2000  ClassM1  ClassM2  ClassM3
538  549-045434  129d48m46.8s  19d43m31.44s        1        0        1
539  549-045435    129d49m12s  19d47m42.72s        1        0        1
795  550-045930    130d03m54s  19d54m54.36s        1        0        1
831  550-045966  130d12m57.6s  19d56m30.48s        1        0        1
897  550-046035  130d33m25.2s  19d50m08.52s        1        0        1

These are triple validation members:

         UCAC4       RAJ2000       DEJ2000  

In [ ]:
	#---------------------------------------------------------------------------------------------------------------------

	def sampedro_match(self, n, dist='0d0m3s', cat='Pan-STARRS',debug=False):
			
		'''
		
		Matches Sampedro catalog with a number (n) of membership approvals with catalog (cat) entries in a radius (dist) given in 'XhYmZs'.

		Output:

		match_idx: list - indices of objects in Pan-STARRS that match with object positions in Sampedro cluster list 

		'''
		
		sampedro=[self.sampedro_n0,self.sampedro_n1,self.sampedro_n2,self.sampedro_n3]
		
		if cat=='Pan-STARRS':
			l=list(self.PS)	
		elif cat == 'K2MASS':
			l=list(self.K2MASS)
		else:
			print('This is not a valid catalog chiffre. I\'ll use Pan-STARRS.')
			l=self.PS
		
		ra2=l[1]
		dec2=l[2]
		l2=sampedro[n]
		ra3=sampedro[n][1] 
		dec3=sampedro[n][2]
		#match Sampedro cluster members with Pan-STARRS
		try:

			match_idx, match_K2ID, match_K2Ra,match_K2Dec,len_match_idx,other_match_ID,match_2mass_ID=match(l,l2,ra2,dec2,ra3,dec3,dist)
		
		except ValueError:
			print('This is ra PS \n{}\n and this is dec PS \n{}\n'.format(ra2[0:5],dec2[0:5]))
			print(cat + 'catalog list or Sampedro_n' + str(n) + ' are probably empty. No matching possible.')
			match_idx=[0]


	
		return match_idx, match_K2ID, match_K2Ra,match_K2Dec,len_match_idx,match_2mass_ID

	#--------------------------------------------------------------------------------------------

	def second_order_match(self, n, distPS='0d0m3s', dist='0d0m3s',debug=False):

		'''
		
		Matching K2 and Pan-STARRS (conditional on Sampedro cluster membership with (n) shared assessments).
		
		'''
		lK2=list(self.K2)
		lPS=list(self.PS)
		print(len(lK2))
		match_idx_PS, match_K2ID, match_K2Ra,match_K2Dec,length,match_2mass_ID=self.sampedro_match(n, dist=distPS, cat='Pan-STARRS',debug=debug)

		idx_K2=list(range(len(lK2[0])))
		
		for _ in range(len(self.K2)):
			lK2[_]= [i for j, i in enumerate(lK2[_]) if j in idx_K2]

		for _ in range(len(self.PS)):
			lPS[_]= [i for j, i in enumerate(lPS[_]) if j in match_idx_PS]
		print(len(lPS[0]))
		print('From the ' + self.name + ' members as assessed by Sampedro '+str(n)+' ' + str(len(lPS[0])) + ' have a Pan-STARRS ID.')
		if len(lPS[1])==0 or len(lK2[1])==0:
			print('The arrays are empty for Sampedro_' + str(n)+'.')
			second_match_idx=[]
		else:
			ra2=lK2[1]
			dec2=lK2[2]
			
			ra3=lPS[1] 
			dec3=lPS[2]

			second_match_idx, match_K2_ID, match_K2_Ra, match_K2_Dec,len_second_match_idx,other_match_ID,match_2mass_ID =match(lK2,lPS,ra3,dec3,ra2,dec2,dist)

		return second_match_idx, match_K2_ID, match_K2_Ra, match_K2_Dec, len_second_match_idx,other_match_ID
		
	#--------------------------------------------------------------------------------------------
	def readselect(self,ids,cat='PS'):
		
		import linecache
		cats={'PS':['_panstarrs_search.txt',[24,32,40,48,56]], '2MASS':'_k2_search.txt'}
		dt=np.unicode_
		line=linecache.getline('cats/'+self.sampedro_name+cats[cat][0],ids)
		print(line)
		myarray = np.fromstring(line,dtype=dt, sep='\t')

		print(myarray)
		mags=[myarray[i] for i in cats[cat][1]]
		return mags

    #--------------------------------------------------------------------------------------------

	def read_params(self, udict):

		for key, value in udict.items():
			#For all Pan-STARRS IDs:
			if type(value)==np.int64:
				ps=list(self.PS)
				ids=ps[0].index(value)
				grizy=self.readselect(ids,cat='PS')
				print(grizy)
			
			#For all 2MASS IDs:
			else:
				k2=list(self.K2)
				ids=k2[3].index(value)
				jhk=self.readselect(ids,cat='2MASS')
				print(jhk)
		return
#------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------

#General purpose funcs:
def match(cat,other_cat,ra1,dec1,ra2,dec2,dist='0d0m1s'):#ra2, dec2 correspond to cat
    '''
    Finds the entries in a catalog cat with corresponsing ra2 and dec2 that match with a list or coordinates (ra1,dec1)  
    '''
    print(ra1[0:10],ra2[0:10],dec1[0:10],dec2[0:10])
    ra1,ra2,dec1,dec2=Angle(ra1,unit=u.deg),Angle(ra2,unit=u.deg),Angle(dec1,unit=u.deg),Angle(dec2,unit=u.deg)
    c = ICRS(ra=ra2, dec=dec2)
    catalog = ICRS(ra=ra1, dec=dec1)
    idx, d2d, d3d = match_coordinates_sky(c, catalog,nthneighbor=1) #idx are indices into catalog that are the closest objects to each of the coordinates in c
    second_match_idx=are_within_bounds(idx, d2d,'0d0m0s', dist)
    other_idx, other_d2d,other_d3d=match_coordinates_sky(catalog,c,nthneighbor=1)
    other_second_match_idx=are_within_bounds(other_idx, other_d2d,'0d0m0s', dist)
	
    match_ID=[i for j, i in enumerate(cat[0]) if j in second_match_idx]
    try:  
    	match_2mass_ID=[i for j, i in enumerate(cat[3]) if j in second_match_idx]
    except IndexError:
        match_2mass_ID=[]
    match_Ra=[i for j, i in enumerate(cat[1]) if j in second_match_idx]
    match_Dec=[i for j, i in enumerate(cat[2]) if j in second_match_idx]
    
    other_match_ID=[i for j, i in enumerate(other_cat[0]) if j in other_second_match_idx]
    return second_match_idx,match_ID,match_Ra, match_Dec, str(len(second_match_idx)), other_match_ID,match_2mass_ID

def mocktest():
	
	x=OpenCluster('mockup','mockup', radius=10, age=1)
	x.loadcatalogs()
	x.refinesampedro()
	
	try:
		l, k2,lengtha=x.sampedro_match(1,dist='0d0m5s',cat='Pan-STARRS')
		l, k2,lengthb=x.sampedro_match(1,dist='0d0m3s',cat='Pan-STARRS')
		l, k2,lengthc=x.sampedro_match(1,dist='0d0m5s',cat='K2MASS')
		l, k2,lengthd=x.sampedro_match(1,dist='0d0m3s',cat='K2MASS')
		l, lengthe=x.second_order_match(1,distPS='0d0m5s',dist='0d0m5s')
		l, lengthf=x.second_order_match(1,distPS='0d0m5s',dist='0d0m3s')
		l, lengthg=x.second_order_match(1,distPS='0d0m3s',dist='0d0m5s')
		l, lengthh=x.second_order_match(1,distPS='0d0m3s',dist='0d0m3s')
	
		a=[len(x.sampedro_n0[0]),len(x.sampedro_n1[0]),len(x.PS[0]),len(x.K2[0]), lengtha,lengthb,lengthc,lengthd,lengthe,lengthf,lengthg,lengthh]
		a=[int(i) for i in a]
		assert a==[40,40,40,20,10,10,2,2,5,5,5,5]
		print('Test success. Green light for matching.')
	except AssertionError:
		print('Your results may not be valid. Test failed. Please check.')		
	return

def find(name, path,debug=False):
    
    '''
    Return the path to a file with name if it exists within the folder in path.
    
    Args:
    name: name of the file one is looking for
    path: path where on suspects to find the file
    debug: generates extra output to the command prompt for debugging
    
    Returns:
    path to file
    '''
    
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

def are_within_bounds(idx,d2d, min_angle, max_angle,debug=False):
	
	'''
	Check if the objects with indices idx which match a catalog object within distance d2d lie within [min_angle, max-angle].
	
	Args:
	idx: list of indices into said catalog for identification
	d2d: list of corresponding distances
	min_angle: minimum distance in dms format
	max_angle: maximum distance in dms format
	debug: generates extra output to the command prompt for debugging
	
	Returns:
	n: indices extracted from idx that fulfil the condition for angular distance
	'''
	
	l=[]
	d=[]
	for _ in range(len(d2d)):	
		if d2d[_].is_within_bounds(min_angle, max_angle): 
			l.append(idx[_])
			d.append(d2d[_])
	
	n=[]
	lpop=list(l)
	dpop=list(d)
	while len(lpop)>0:
		#find same indices
		m=[i for i, j in enumerate(lpop) if j == lpop[0]]
		if len(m)>1:
			for k in range(len(m)-1):
				if dpop[m[k]].is_within_bounds(dpop[m[k+1]]): 
					minidx=lpop[m[k]]
				else: 
					minidx=lpop[m[k+1]]
		else: minidx=lpop[m[0]]
		n.append(minidx)
		lpop=[i for j, i in enumerate(lpop) if j not in m]
		dpop=[i for j, i in enumerate(dpop) if j not in m]
	return n


#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------


def wrap_cross(inputs,debug=False):
	for item in inputs:
		x=OpenCluster(item[0],item[1], radius=item[2], age=item[3])
		x.loadcatalogs(debug=debug)
		print('\nMatching catalogs for ' + x.name + ':\n')
		x.refinesampedro(debug=debug)
		#with        
		#out.write(str(len(x.sampedro_n0[0]))+'\n'+str(len(x.sampedro_n1[0]))+'\n'+str(len(x.sampedro_n2[0]))+'\n'+str(len(x.sampedro_n3[0]))+'\n'+str(len(x.PS[0]))+'\n'+str(len(x.K2[0]))+'\n')
		lps_m, ps_m, psra_m,psdec_m,len1,PS_match_ID=x.second_order_match(1,distPS='0d0m3s',dist='0d0m3s',debug=debug)
		lk2_m, k2_m,k2ra_m,k2dec_m, len2,twomass_match_ID=x.sampedro_match(1,dist='0d0m3s',cat='K2MASS',debug=debug)
		inter1=open(x.sampedro_name + '_inter1.txt', 'w')
		inter2=open(x.sampedro_name + '_inter2.txt', 'w')
		for a,b in zip(ps_m,PS_match_ID):
			inter1.write(str(a)+','+str(b)+'\n')
		inter1.close()		
		for a,b in zip(k2_m,twomass_match_ID):
			inter2.write(str(a)+','+str(b)+'\n')
		inter2.close()
	return x.PS, x.K2, x



#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------

def wrap_basic(inputs,debug=False):
	for item in inputs:
		
		x=OpenCluster(item[0],item[1], radius=item[2], age=item[3])
		x.loadcatalogs(debug=debug)
		print('\nMatching catalogs for ' + x.name + ':\n')
		x.refinesampedro(debug=debug)
		out=open('share/'+item[1]+'_IDs.txt', 'w')
		out.write(str(len(x.sampedro_n0[0]))+'\n'+str(len(x.sampedro_n1[0]))+'\n'+str(len(x.sampedro_n2[0]))+'\n'+str(len(x.sampedro_n3[0]))+'\n'+str(len(x.PS[0]))+'\n'+str(len(x.K2[0]))+'\n')
		

		#all line numbers +2 in parameters sheet
		for i in range(1,4):
			##10
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m5s',cat='Pan-STARRS',debug=debug)
			out.write(length+'\n')
			##11
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m3s',cat='Pan-STARRS',debug=debug)
			out.write(length+'\n')
		for i in range(1,4): 
			##16,18,20
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m5s',cat='K2MASS',debug=debug)
			out.write(length+'\n')
			##17,19,21
			l, k2, ra,dec,length=x.sampedro_match(i,dist='0d0m3s',cat='K2MASS',debug=debug)
			out.write(length+'\n')
		
			# if i==1:		
			#     for item in ps:
			#         ids.write(str(item)+'\n')
		for i in range(1,4):
			##22,26,30
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m5s',dist='0d0m5s',debug=debug)
			out.write(length+'\n')
			##23,27,31
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m5s',dist='0d0m3s',debug=debug)
			out.write(length+'\n')
			##24,28,32
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m3s',dist='0d0m5s',debug=debug)
			out.write(length+'\n')
			##25,29,33
			l, ps, ra,dec,length,other_match_ID=x.second_order_match(i,distPS='0d0m3s',dist='0d0m3s',debug=debug)
			out.write(length+'\n')
			
			#out.write(length+'\n')
		out.close()
	return

In [85]:
def wrap_further(X):
    ps_m,PS_match_ID=np.loadtxt(X.sampedro_name + '_inter1.txt', delimiter=',', unpack=True,dtype=[('ID','i8'),('PS', 'i8')])
    k2_m,twomass_match_ID=np.loadtxt(X.sampedro_name + '_inter2.txt', delimiter=',', unpack=True,dtype=[('ID','i8'),('2MASS',np.unicode_,18)])
    ps_m,PS_match_ID,k2_m,twomass_match_ID=list(ps_m),list(PS_match_ID),list(k2_m),list(twomass_match_ID)
    
    ps = pd.read_csv(X.sampedro_name + '_inter1.txt', 
                     delimiter=',',
                     names=['EPIC','PSID']
                    )
    k2 = pd.read_csv(X.sampedro_name + '_inter2.txt', 
                     delimiter=',',
                     names=['EPIC','2MASS']
                    )
    print(ps.head(),'\n',k2.head())

    PSdict=dict(zip(ps_m,PS_match_ID))
    TwoMASSdict=dict(zip(k2_m,twomass_match_ID))
    cross=set(ps_m) & set(k2_m)
    union=set(ps_m) | set(k2_m)
    union_dict = TwoMASSdict
    union_dict.update(PSdict)
    #print(PSdict)
    #print(union_dict)
    #print(len(union))
    #print(type(PS_match_ID[0]))

    udict_color=read_params2(PS,X,union_dict)
    
    with open('share/cross_match/union/'+X.sampedro_name+'_parameter.txt', 'w') as udict_out:
        udict_out.write(X.sampedro_name + ', ' + str(X.radius) + ' arcmin, ' + str(X.age) + ' Gyr, ' + str(X.distance) + ' pc\n')
        for key, value in udict_color.items():
            color=''
            if value[1] == 'grizyJ':
                for i,col in enumerate(value[2]):
                    color=color+','+str(col)
                udict_out.write(str(key)+','+str(value[0])+','+str(value[1])+color+',NaN,NaN\n')
            elif value[1] == 'JHKJ':
                    for i,col in enumerate(value[2][:-1]):
                        color=color+','+str(col)
                        
                    udict_out.write(str(key)+','+str(value[0])+','+str(value[1])+',NaN,NaN,NaN,NaN,NaN'+color+'\n')



    #Objects with 2MASS and Pan-STARRS respectively
    id2mass=open('share/2mass_match/'+X.sampedro_name+'_2mass_IDs.txt', 'w')
    idps=open('share/panstarrs_match/'+X.sampedro_name+'_panstarrs_IDs.txt', 'w')
    for i in k2_m:
        id2mass.write('EPIC '+str(i)+'\n')

    for i in ps_m:
        idps.write('EPIC '+str(i)+'\n')
    idps.close()
    id2mass.close()


    #Cross section of Pan-STARRS and 2MASS
    idcross=open('share/cross_match/'+X.sampedro_name+'_cross_IDs.txt', 'w')
    #Union of Pan-STARRS and 2MASS		
    idunion=open('share/cross_match/union/'+X.sampedro_name+'_union_IDs.txt', 'w')	
    for i in list(cross):
        idcross.write('EPIC '+str(i)+'\n')

    for i in union:
        idunion.write('EPIC '+str(i)+'\n')

    idcross.close()
    idunion.close()
    return

In [106]:
import pandas as pd
def readselect2(PS,K2,obj,ids,cat='PS'):

    import linecache
    cats={'PS':['_panstarrs_search.txt',[24,32,40,48,56]]}
    dt=np.unicode_
    line=linecache.getline('cats/'+obj.sampedro_name+cats[cat][0],ids)
    #print(line)
    myarray=np.core.defchararray.split(line, sep="\t").tolist()
    
    
    #print(myarray)
    mags=[float(myarray[i]) for i in cats[cat][1]]
    
    return mags
#--------------------------------------------------------------------------------------------

def read_params2(PS,obj,udict):
    k2=np.genfromtxt('share/cross_match/union/'+obj.sampedro_name + '_k2_JHK.csv',
                     delimiter=',',
                     skip_header=2,
                     usecols=(0,20,27,28,29),dtype=[('k2ID','U12'),('ID','U16'),('J','f8'),('H','f8'),('K','f8')])
    df = pd.read_csv('share/cross_match/union/'+obj.sampedro_name + '_k2_JHK.csv',
                     delimiter=',',
                     skiprows =[1],
                     usecols=['K2 ID','2MASS ID','J Mag','H Mag','K Mag'],
                    )
    ps = pd.read_csv('cats/'+obj.sampedro_name + '_panstarrs_search.txt',
                     delimiter=',',
                     skiprows =[1],
                     usecols=['K2 ID','2MASS ID','J Mag','H Mag','K Mag'],
                    )
    df.rename(index=int,
              columns={'K2 ID': 'EPIC','2MASS ID':'2MASS','J Mag' : 'J','H Mag' : 'H','K Mag': 'K'},
              inplace=True
             )
    #ps = pd.DataFrame(list(PS),columns=['PSID'])
    print('\n',df.head())
    #J=k2['J'].tolist()
    #H=k2['H'].tolist()
    #K=k2['K'].tolist()
    #k2mass=k2['ID'].tolist()
    #k2ID=k2['k2ID'].tolist()
    #k2ID=[float(item) for item in k2ID]
  
    #ps=list(PS)
    u={key : [udict[key]] for key in udict}
    union = pd.DataFrame.from_dict(u).T
    union.rename(index=int, columns={0 : 'id'},inplace=True)
    print(union.head())
    udict={}
    for epic, id_ in union.iterrows():
        print(type(id_.id))
        #For all Pan-STARRS IDs:
        if type(id_.id) == int:

            #ids=ps[0].index(value)
            grizy=readselect2(PS,K2,obj,epic,cat='PS')
            grizy.append(df.J[df.EPIC==epic])
            #print(grizy)
            
            udict[epic]=[id_,'grizyJ',grizy]
            

        #For all 2MASS IDs:
        else:
            id_=str(id_)
            for ids, row in df.iterrows():
                if row['2MASS'] == id_:
                    jhk=[df.J[ids],df.H[ids],df.K[ids],df.J[ids]]
                    #print(jhk)
                    udict[epic]=[id_,'JHKJ',jhk]
    print(udict)
    return udict

In [107]:
inputs=[]
#inputs.append(['M67','M67', 15, 4.0])
#inputs.append(['Ruprecht 147','Ruprecht_147', 30, 2.5, 300.])
inputs.append(['M44','M44', 47, 0.73, 181.5])
#PS,K2,X=wrap_cross(inputs,debug=True)
print(type(X))
wrap_further(X)

<class '__main__.OpenCluster'>
        EPIC                PSID
0  211970750  131601300919634874
1  211969037  131601300969998757
2  211956096  131621301088294267
3  211974702  131641300519016934
4  211968107  131561300512507952 
         EPIC             2MASS
0  211960587  08403347+1938009
1  211959779  08400968+1937170
2  211968228  08401378+1944559
3  211968762  08400472+1945252
4  211970147  08401345+1946436

         EPIC       J             2MASS       H       K
0  211960587  10.706  08403347+1938009  10.262  10.169
1  211959779  10.556  08400968+1937170  10.240  10.133
2  211968598  13.878  08401884+1945158  13.557  13.497
3  211968228  12.752  08401378+1944559  12.065  11.881
4  211958885  15.774  08403799+1936285  15.078  14.858
                           id
211917139  131801304076509874
211918994  131981301921321207
211920603  131251302241635164
211920741  131971299405177020
211921414  131481304393998059
<class 'int'>


IndexError: list index out of range

In [9]:
print(type())

<class '__main__.OpenCluster'>
